In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.4
numpy 1.19.5
pandas 1.1.5
sklearn 0.24.2
tensorflow 2.6.2
keras.api._v2.keras 2.6.0


In [2]:
# constant是常量张量
t = tf.constant([[1., 2., 3.], [4., 5.,6.]])

# index
# 2.0能够直接获取值是因为execution默认是打开的
print(t)
print(t[:, 1:])
print(t[..., 1])  # ...效果与:类似，代表所有
# t.assign(1)对常量不能进行再次assign设置
print(type(t.numpy()))  # 转为ndarray
q = t.numpy()
q

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
<class 'numpy.ndarray'>


array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [3]:
t1 = tf.constant(q)  # 把ndarray变为张量
t1

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
# ops 使用tf本身的math接口对Tensor进行计算（张量运算）
print(t+10)
print(t)  # 不是就地改变
print(tf.square(t))
print(t)
# 矩阵乘以自己的转置
print(tf.transpose(t))
print(t @ tf.transpose(t))  # @是矩阵乘法，和*不一致

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 4.]
 [2. 5.]
 [3. 6.]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [5]:
print(tf.sqrt(t))  # 求平方根
print('-'*50)
# tf.math.sqrt(t)
tf.math.log(t)  # 必须加math

tf.Tensor(
[[1.        1.4142135 1.7320508]
 [2.        2.236068  2.4494898]], shape=(2, 3), dtype=float32)
--------------------------------------------------


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.       , 0.6931472, 1.0986123],
       [1.3862944, 1.609438 , 1.7917595]], dtype=float32)>

In [6]:
# numpy conversion
print(t.numpy())  # 可以直接通过numpy取出来
print(t.numpy().tolist())
print(type(t.numpy()))
print('-' * 50)
print(np.square(t)) # 直接求平方
np_t = np.array([[1., 2., 3.], [4., 5., 6.]])
print(tf.constant(np_t)) # 转换为tensor

[[1. 2. 3.]
 [4. 5. 6.]]
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
<class 'numpy.ndarray'>
--------------------------------------------------
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [7]:
# Scalars标量：只有一个数值的张量，称为标量
t = tf.constant(2.718)
print(t)
print(t.numpy())
print(t.shape)  # 维数

tf.Tensor(2.718, shape=(), dtype=float32)
2.718
()


In [8]:
# strings
t = tf.constant("cafe")
print(t)  # b:二进制
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [9]:
# string array
t = tf.constant(["cafe", "coffee", "咖啡"])
# 自动求出数组中每一个字符的长度,如果不加unit="UTF8_CHAR"，得到的是实际字节存储的长度
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.length(t))  # 一个汉字3个字节存储
print('-' * 50)
print(tf.strings.length(t, unit="BYTE"))  
r = tf.strings.unicode_decode(t, "UTF8")
# https://tool.chinaz.com/tools/unicode.aspx  汉字转的是unicode编码
print(r)


tf.Tensor([4 6 2], shape=(3,), dtype=int32)
tf.Tensor([4 6 6], shape=(3,), dtype=int32)
--------------------------------------------------
tf.Tensor([4 6 6], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [10]:
# RaggedTensor 是指形状分布不固定的（行元素个数不相等）-ragged参差不齐的
# Tensor，2.0新增
r = tf.ragged.constant([[11, 12], [21, 22, 23], [], [41]])

# index op
print(r)
print(r.shape)  # (4, None)
print(r[1])
# 取一行也是ragged tensor
print(r[1:3])
# print(r[:,1])#不能取列索引

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
(4, None)
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23], []]>


In [13]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51, 52],[], [], [71]])
# r2 = tf.ragged.constant([[51, 52], [], [71]])
print(tf.concat([r, r2], axis = 0))
print(tf.concat([r, r2], axis = 1))  # 行数不相等，不可以拼

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [], [71]]>
<tf.RaggedTensor [[11, 12, 51, 52], [21, 22, 23], [], [41, 71]]>


In [14]:
# 按轴1进行合并时，行数要一致，行数相等才可以拼
r3 = tf.ragged.constant([[13, 14], [15], [41], [42, 43]])
print(tf.concat([r, r3], axis = 1))

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [41], [41, 42, 43]]>


In [15]:
print(r.to_tensor())  # 各种深度学习模型必须输入一个tensor
# 空闲的补0，只能往后面补

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [16]:
# sparse tensor 可以往前面补零,sparse tensor从第一行依次往下填位置
# sparese tensor存储节省内存空间，磁盘空间
s = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3],[3,2]], # 位置
                    values = [1., 2., 3.,5],  # 值
                    dense_shape = [4, 4])  # 维数
print(s)
tt = tf.sparse.to_dense(s)
tt

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]
 [3 2]], shape=(4, 2), dtype=int64), values=tf.Tensor([1. 2. 3. 5.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.],
       [0., 0., 5., 0.]], dtype=float32)>

In [19]:
# ops on sparse tensors

s2 = s * 2.0
print(s2)
print('-' * 50)

# 不支持加法
try:
    s3 = s + 1.0
#     s3 = s + 1
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [50., 60.],
                  [70., 80.]])
# tf.sparse.to_dense(s)@s4
print(tf.sparse.sparse_dense_matmul(s, s4))  # 稀疏Tensor和Tensor相乘

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]
 [3 2]], shape=(4, 2), dtype=int64), values=tf.Tensor([ 2.  4.  6. 10.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([4 4], shape=(2,), dtype=int64))
--------------------------------------------------
unsupported operand type(s) for +: 'SparseTensor' and 'float'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]
 [250. 300.]], shape=(4, 2), dtype=float32)


In [20]:
# sparse tensor
s5 = tf.SparseTensor(indices = [[0, 2], [2, 3], [0, 1]],
                    values = [1., 2., 3.],
                    dense_shape = [3, 4])
# print(tf.sparse.to_dense(s5))  # sparse无顺序时，不能转为tensor，会报错
print(s5)
s6 = tf.sparse.reorder(s5)
print(s6)
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [2 3]
 [0 1]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
SparseTensor(indices=tf.Tensor(
[[0 1]
 [0 2]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([3. 1. 2.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 3. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]], shape=(3, 4), dtype=float32)


In [26]:
# Variables
v = tf.Variable([[1., 2., 3.], [4., 5.,6.]])
print(v)
print(v.value())
print('-'*50)
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
--------------------------------------------------
[[1. 2. 3.]
 [4. 5. 6.]]


In [27]:
# 修改变量时要用assign，改变tensor内某个值，空间没有发生变化，效率高
# assign value
print(id(v))
v.assign(2*v)
print(id(v))
print(v.numpy())
print('-'*50)
v[0, 1].assign(42)  # 取某个元素修改
print(v.numpy())
print('-'*50)
v[1].assign([7., 8., 9.])  # 取某一行修改
print(v.numpy())
print(id(v))

139673618129864
139673618129864
[[ 2.  4.  6.]
 [ 8. 10. 12.]]
--------------------------------------------------
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
--------------------------------------------------
[[ 2. 42.  6.]
 [ 7.  8.  9.]]
139673618129864


In [28]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


In [29]:
v = 2 * v  # 重新分配空间
print(v)
print(id(v))
print(type(v))

tf.Tensor(
[[ 4. 84. 12.]
 [14. 16. 18.]], shape=(2, 3), dtype=float32)
139673618103816
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [33]:
x = tf.constant([[1., 6.], [3., 2.]])
print(x)
tf.reduce_mean(x,axis=1)  # Computes the mean of elements across dimensions of a tensor

tf.Tensor(
[[1. 6.]
 [3. 2.]], shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([3.5, 2.5], dtype=float32)>